In [1]:
# ! pipenv run jupyter nbconvert --to python pull_l2_activity.ipynb

In [2]:
# import os

# # Print all environment variables
# for key, value in os.environ.items():
#     print(f"{key}: {value}")

In [3]:
print("start l2 activity")
import sys

sys.path.append("../helper_functions")
import duneapi_utils as d
import growthepieapi_utils as gtp
import l2beat_utils as ltwo
import csv_utils as cu
import google_bq_utils as bqu
import pandas_utils as pu
import clickhouse_utils as ch
import opstack_metadata_utils as ops

sys.path.pop()

import numpy as np
import pandas as pd
import time
from datetime import datetime, timezone

start l2 activity


In [4]:
# L2Beat removed metadata files, comment out

print("getting metadata - l2beat")
l2beat_meta = ltwo.get_l2beat_metadata()
l2beat_meta["chain"] = l2beat_meta["slug"]
# l2beat_meta["is_upcoming"] = l2beat_meta["is_upcoming"].fillna(False)

getting metadata - l2beat
Failed to fetch metadata from GitHub API: 401 Client Error: Unauthorized for url: https://api.github.com/repos/l2beat/l2beat/contents/packages/config/src/projects
This is likely due to authentication issues or L2Beat repository changes.
Returning minimal metadata DataFrame.
No metadata files found. Returning minimal DataFrame with basic chain information.


In [5]:
l2beat_meta

,layer,slug,file_name,chainId,name,explorerUrl,rpcUrl,category,provider,hostChain,da_provider_name,badges,is_upcoming,is_archived,is_current_chain,websites,documentation,repositories,provider_entity,chain
0,L2,unknown,unknown,None,Unknown Chain,None,None,None,None,None,None,None,False,False,True,None,None,None,None,unknown


In [6]:
# L2B Meta
l2b_summary = ltwo.get_l2beat_chain_summary()
l2b_summary['dt'] = pd.to_datetime(datetime.now(timezone.utc).date())

Unexpected response structure from L2Beat API
Response keys: ['chart', 'projects']


In [7]:
# display(l2b_summary.head(5))
# print(l2b_summary.dtypes)

# bqu.append_and_upsert_df_to_bq_table(
#     l2b_summary,
#     "daily_l2beat_chain_summary",
#     unique_keys=["dt", "id","slug"],
# )

In [8]:
# # # Usage
try:
    gtp_api = gtp.get_growthepie_api_data()
    gtp_meta_api = gtp.get_growthepie_api_meta()
    gtp_api = gtp_api.rename(columns={"date": "dt"})
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Exception type: {type(e).__name__}")
    import traceback
    print("Traceback:")
    print(traceback.format_exc())

https://api.growthepie.xyz/v1/fundamentals.json


In [9]:
# gtp_api.sort_values(by='dt', ascending=False)

In [ ]:
# update datatype for bq uploads
if "enable_contracts" in gtp_meta_api:
    gtp_meta_api["enable_contracts"] = gtp_meta_api["enable_contracts"].astype(bool)

if "colors" in gtp_meta_api:
    gtp_meta_api["colors"] = gtp_meta_api["colors"].astype(str)

# ensure 'active_tabs' exists to match schema (fallback to 'tab_status' or empty)
if "active_tabs" not in gtp_meta_api.columns:
    if "tab_status" in gtp_meta_api.columns:
        gtp_meta_api["active_tabs"] = gtp_meta_api["tab_status"]
    else:
        gtp_meta_api["active_tabs"] = ""
# cast to string for BQ compatibility
if "active_tabs" in gtp_meta_api.columns:
    gtp_meta_api["active_tabs"] = gtp_meta_api["active_tabs"].astype(str)

In [11]:
print("getting assets onchain - l2beat")
# l2beat_aoc = ltwo.get_daily_aoc_by_token()
# l2beat_aoc = l2beat_aoc.rename(columns={"project": "chain", "date": "dt"})
time.sleep(3)  # rate limits

getting assets onchain - l2beat


In [12]:
print("getting data - l2beat")
# l2beat_df = ltwo.get_all_l2beat_data()

getting data - l2beat


In [13]:
# l2beat_meta[l2beat_meta['chainId'] == '324']
# l2beat_meta

In [14]:
# combined_l2b_df = l2beat_df.merge(
#     l2beat_meta[
#         [
#             "chain",
#             "name",
#             "layer",
#             "chainId",
#             "provider",
#             "provider_entity",
#             "category",
#             "is_upcoming",
#             "is_archived",
#             "is_current_chain",
#         ]
#     ],
#     on="chain",
#     how="outer",
# )

# combined_l2b_df["chainId"] = combined_l2b_df["chainId"].astype("Int64")

In [15]:
print("getting data - growthepie")
try:
    if not gtp_api.empty and not gtp_meta_api.empty:
        combined_gtp_df = gtp_api.merge(
            gtp_meta_api[["origin_key", "chain_name", "evm_chain_id"]],
            on="origin_key",
            how="left",
        )
        combined_gtp_df["dt"] = pd.to_datetime(combined_gtp_df["dt"], errors="coerce")

        combined_gtp_df = combined_gtp_df.drop(columns=("index"))
        # combined_gtp_df.sample(5)
    else:
        print("GrowThePie API data is empty, skipping merge")
        combined_gtp_df = pd.DataFrame()
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Exception type: {type(e).__name__}")
    import traceback
    print("Traceback:")
    print(traceback.format_exc())
    combined_gtp_df = pd.DataFrame()

getting data - growthepie


In [16]:
# Check Columns
try:
    if not combined_gtp_df.empty:
        column_names = combined_gtp_df.columns

        # If we have *_usd columns, ensure matching *_eth columns exist
        for col in column_names:
            if col.endswith("_usd"):
                new_col_name = col.replace("_usd", "_eth")
                if new_col_name not in combined_gtp_df.columns:
                    combined_gtp_df[new_col_name] = np.nan

        # Some columns are provided without a "_usd" suffix but schema expects *_eth pairs
        required_base_cols = ["tvl", "stables_mcap"]
        for base in required_base_cols:
            eth_col = f"{base}_eth"
            if base in combined_gtp_df.columns and eth_col not in combined_gtp_df.columns:
                combined_gtp_df[eth_col] = np.nan
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Exception type: {type(e).__name__}")
    import traceback
    print("Traceback:")
    print(traceback.format_exc())

In [17]:
# Add Metadata
opstack_metadata = ops.get_op_stack_metadata_df()
# combined_l2b_df["l2beat_slug"] = combined_l2b_df["chain"]
# meta_cols = [
#     "l2beat_slug",
#     "is_op_chain",
#     "mainnet_chain_id",
#     "op_based_version",
#     "alignment",
#     "chain_name",
#     "display_name",
# ]

# l2b_enriched_df = combined_l2b_df.merge(
#     opstack_metadata[meta_cols], on="l2beat_slug", how="left"
# )

# l2b_enriched_df["alignment"] = l2b_enriched_df["alignment"].fillna("Other EVMs")

In [18]:
boolean_columns = ["is_op_chain", "is_upcoming", "is_archived", "is_current_chain"]
dfs = [l2beat_meta]#[l2b_enriched_df, l2beat_meta]

for df in dfs:
    for column in boolean_columns:
        if column in df.columns:
            df[column] = df[column].fillna(False)

In [19]:
#  Define aggregation functions for each column
aggregations = {
    "totalUsd": ["min", "last", "mean"],  # valueUsd
    "transactions": ["sum", "mean"],
    "canonicalUsd": ["min", "last", "mean"],  # cbvUsd
    "externalUsd": ["min", "last", "mean"],  # ebvUsd
    "nativeUsd": ["min", "last", "mean"],  # nmvUsd
}


# Function to perform aggregation based on frequency
def aggregate_data(df, freq, date_col="timestamp", groupby_cols=None, aggs=None):
    if groupby_cols is None:
        groupby_cols = [
            "chain",
            "chainId",
            "layer",
            "is_op_chain",
            "mainnet_chain_id",
            "op_based_version",
            "alignment",
            "chain_name",
            "display_name",
            "provider",
            "is_upcoming",
            "is_archived",
            "is_current_chain",
        ]
    if aggs is None:
        aggs = aggregations

    # Group by the specified frequency and other columns, then apply aggregations
    df_agg = (
        df.groupby([pd.Grouper(key=date_col, freq=freq)] + groupby_cols, dropna=False)
        .agg(aggs)
        .reset_index()
    )

    # Flatten the hierarchical column index and concatenate aggregation function names with column names
    df_agg.columns = ["_".join(filter(None, col)).rstrip("_") for col in df_agg.columns]

    # Rename the 'timestamp' column based on the frequency
    date_col_name = "month" if freq == "MS" else "week"
    df_agg.rename(columns={date_col: date_col_name}, inplace=True)

    # Group by 'chain' and rank the rows within each group based on the date column
    df_agg[f"{date_col_name}s_live"] = df_agg.groupby("chain")[date_col_name].rank(
        method="min"
    )

    return df_agg


# # Perform monthly aggregation
# l2b_monthly_df = aggregate_data(l2b_enriched_df, freq="MS")
# # Perform weekly aggregation
# l2b_weekly_df = aggregate_data(l2b_enriched_df, freq="W-MON")

# Sample output
# l2b_weekly_df.sample(5)

In [20]:
# export
folder = "outputs/"
# combined_gtp_df.to_csv(folder + "growthepie_l2_activity.csv", index=False)
# gtp_meta_api.to_csv(folder + "growthepie_l2_metadata.csv", index=False)
# l2b_enriched_df.to_csv(folder + "l2beat_l2_activity.csv", index=False)
# l2beat_meta.to_csv(folder + "l2beat_l2_metadata.csv", index=False)
# l2b_monthly_df.to_csv(folder + "l2beat_l2_activity_monthly.csv", index=False)
# l2b_weekly_df.to_csv(folder + "l2beat_l2_activity_weekly.csv", index=False)
# l2beat_aoc.to_csv(folder + "l2beat_aoc_by_token.csv", index=False)

In [21]:
# # gtp_meta_api.dtypes
# gtp_meta_api['logo'] = gtp_meta_api['logo'].astype(str)
# gtp_meta_api['l2beat_stage'] = gtp_meta_api['l2beat_stage'].astype(str)
# # gtp_meta_api['stack'] = gtp_meta_api['stack'].astype(str)
# gtp_meta_api['block_explorers'] = gtp_meta_api['block_explorers'].astype(str)

In [22]:
# gtp_meta_api

In [23]:
# combined_gtp_df['evm_chain_id'] = combined_gtp_df['evm_chain_id'].astype(str).replace('.0','').fillna('-')
# gtp_meta_api['evm_chain_id'] = gtp_meta_api['evm_chain_id'].astype(str).replace('.0','').fillna('-')

In [24]:
combined_gtp_df.sort_values(by='dt', ascending=False)

,origin_key,dt,aa_last7d,app_fees_usd,costs_blobs_usd,costs_l1_usd,costs_total_usd,daa,fdv_usd,fees_paid_usd,...,costs_l1_eth,costs_total_eth,fdv_eth,fees_paid_eth,market_cap_eth,profit_eth,rent_paid_eth,txcosts_median_eth,tvl_eth,stables_mcap_eth
2995,zora,2025-11-03,NaN,NaN,NaN,NaN,NaN,NaN,6.783436e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
907,imx,2025-11-03,NaN,NaN,NaN,NaN,NaN,NaN,1.019553e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2722,unichain,2025-11-03,NaN,NaN,NaN,NaN,NaN,NaN,5.861856e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
634,ethereum,2025-11-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2813,worldchain,2025-11-03,NaN,NaN,NaN,NaN,NaN,NaN,8.421315e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,linea,2025-08-05,121068.0,3.258809e+04,4.091013e-08,16.138379,16.138379,22651.0,NaN,4282.896255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
635,fraxtal,2025-08-05,1350.0,9.590000e+02,NaN,24.308326,24.308326,403.0,2.926409e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2632,unichain,2025-08-05,40389.0,2.616902e+05,3.267941e-07,225.918355,225.918356,11506.0,9.941818e+09,17286.409511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2178,scroll,2025-08-05,21829.0,8.357274e+03,3.165665e-08,16.515600,16.515600,4846.0,3.022961e+08,245.796331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# BQ Upload
try:
    bqu.write_df_to_bq_table(l2beat_meta, "l2beat_l2_metadata")
    # time.sleep(1)
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Exception type: {type(e).__name__}")
    import traceback
    print("Traceback:")
    print(traceback.format_exc())

# )

2025-11-03 16:53:51,453 INFO root Attempt 1: Using service account credentials
2025-11-03 16:53:51,454 WARNING root Service account init failed on attempt 1: [Errno 2] No such file or directory: '/Users/michaelsilberling/Documents/bq/oplabs-tools-data-0842f34ceb69.json'
2025-11-03 16:53:51,454 INFO root Attempt 1: Using ADC/OIDC login


Start Writing api_table_uploads.l2beat_l2_metadata
DataFrame columns: {'category', 'is_upcoming', 'websites', 'documentation', 'is_current_chain', 'layer', 'is_archived', 'explorerUrl', 'chainId', 'name', 'badges', 'provider_entity', 'slug', 'file_name', 'repositories', 'chain', 'provider', 'rpcUrl', 'hostChain', 'da_provider_name'}
Schema columns: {'category', 'is_upcoming', 'websites', 'documentation', 'is_current_chain', 'layer', 'is_archived', 'explorerUrl', 'chainId', 'name', 'badges', 'provider_entity', 'slug', 'isArchived', 'file_name', 'repositories', 'chain', 'provider', 'rpcUrl', 'hostChain', 'da_provider_name'}
Error on attempt 1: Schema mismatch. Missing: {'isArchived'}, Extra: set()
Waiting before retry...


2025-11-03 16:53:53,683 INFO root Attempt 1: Using service account credentials
2025-11-03 16:53:53,684 WARNING root Service account init failed on attempt 1: [Errno 2] No such file or directory: '/Users/michaelsilberling/Documents/bq/oplabs-tools-data-0842f34ceb69.json'
2025-11-03 16:53:53,685 INFO root Attempt 1: Using ADC/OIDC login


DataFrame columns: {'category', 'is_upcoming', 'websites', 'documentation', 'is_current_chain', 'layer', 'is_archived', 'explorerUrl', 'chainId', 'name', 'badges', 'provider_entity', 'slug', 'file_name', 'repositories', 'chain', 'provider', 'rpcUrl', 'hostChain', 'da_provider_name'}
Schema columns: {'category', 'is_upcoming', 'websites', 'documentation', 'is_current_chain', 'layer', 'is_archived', 'explorerUrl', 'chainId', 'name', 'badges', 'provider_entity', 'slug', 'isArchived', 'file_name', 'repositories', 'chain', 'provider', 'rpcUrl', 'hostChain', 'da_provider_name'}
Error on attempt 2: Schema mismatch. Missing: {'isArchived'}, Extra: set()
Waiting before retry...


2025-11-03 16:53:56,564 INFO root Attempt 1: Using service account credentials
2025-11-03 16:53:56,565 WARNING root Service account init failed on attempt 1: [Errno 2] No such file or directory: '/Users/michaelsilberling/Documents/bq/oplabs-tools-data-0842f34ceb69.json'
2025-11-03 16:53:56,566 INFO root Attempt 1: Using ADC/OIDC login


DataFrame columns: {'category', 'is_upcoming', 'websites', 'documentation', 'is_current_chain', 'layer', 'is_archived', 'explorerUrl', 'chainId', 'name', 'badges', 'provider_entity', 'slug', 'file_name', 'repositories', 'chain', 'provider', 'rpcUrl', 'hostChain', 'da_provider_name'}
Schema columns: {'category', 'is_upcoming', 'websites', 'documentation', 'is_current_chain', 'layer', 'is_archived', 'explorerUrl', 'chainId', 'name', 'badges', 'provider_entity', 'slug', 'isArchived', 'file_name', 'repositories', 'chain', 'provider', 'rpcUrl', 'hostChain', 'da_provider_name'}
Error on attempt 3: Schema mismatch. Missing: {'isArchived'}, Extra: set()
All attempts failed
An error occurred: Schema mismatch. Missing: {'isArchived'}, Extra: set()
Exception type: ValueError
Traceback:
Traceback (most recent call last):
  File "/var/folders/nl/qcdk9wr515b2h3s1kd4z08w80000gn/T/ipykernel_73475/4389974.py", line 3, in <module>
    bqu.write_df_to_bq_table(l2beat_meta, "l2beat_l2_metadata")
  File "/

In [26]:
try:
    if not combined_gtp_df.empty:
        bqu.write_df_to_bq_table(combined_gtp_df, "daily_growthepie_l2_activity")
        time.sleep(1)
    if not gtp_meta_api.empty:
        bqu.write_df_to_bq_table(gtp_meta_api, "growthepie_l2_metadata")
        time.sleep(1)
# bqu.write_df_to_bq_table(l2b_enriched_df, "daily_l2beat_l2_activity")
# time.sleep(1)
# bqu.write_df_to_bq_table(l2b_monthly_df, "monthly_l2beat_l2_activity")
# time.sleep(1)
# bqu.write_df_to_bq_table(l2b_weekly_df, "weekly_l2beat_l2_activity")
# time.sleep(1)
# bqu.append_and_upsert_df_to_bq_table(
#     l2beat_aoc,
#     "daily_l2beat_aoc_by_token",
#     unique_keys=["dt", "chain", "token_type", "asset_id", "chain", "address"],
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Exception type: {type(e).__name__}")
    import traceback
    print("Traceback:")
    print(traceback.format_exc())

2025-11-03 16:54:06,644 INFO root Attempt 1: Using service account credentials
2025-11-03 16:54:06,645 WARNING root Service account init failed on attempt 1: [Errno 2] No such file or directory: '/Users/michaelsilberling/Documents/bq/oplabs-tools-data-0842f34ceb69.json'
2025-11-03 16:54:06,645 INFO root Attempt 1: Using ADC/OIDC login


Start Writing api_table_uploads.daily_growthepie_l2_activity


/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Chunk 1/1 loaded successfully to api_table_uploads.daily_growthepie_l2_activity
All data loaded successfully to api_table_uploads.daily_growthepie_l2_activity


2025-11-03 16:54:11,552 INFO root Attempt 1: Using service account credentials
2025-11-03 16:54:11,552 WARNING root Service account init failed on attempt 1: [Errno 2] No such file or directory: '/Users/michaelsilberling/Documents/bq/oplabs-tools-data-0842f34ceb69.json'
2025-11-03 16:54:11,553 INFO root Attempt 1: Using ADC/OIDC login


Start Writing api_table_uploads.growthepie_l2_metadata
DataFrame columns: {'technology', 'excluded_metrics', 'gas_token', 'twitter', 'rhino_naming', 'supported_metrics', 'l2beat_id', 'website', 'colors', 'symbol', 'similar_chains', 'links', 'logo', 'evm_chain_id', 'rhino_listed', 'tab_status', 'description', 'bucket', 'chain_name', 'stack', 'raas', 'name', 'block_explorer', 'l2beat_stage', 'origin_key', 'enable_contracts', 'maturity', 'da_layer', 'l2beat_link', 'block_explorers', 'url_key', 'deployment', 'chain_type', 'ecosystem', 'caip2', 'company', 'launch_date', 'purpose'}
Schema columns: {'technology', 'twitter', 'excluded_metrics', 'gas_token', 'rhino_naming', 'supported_metrics', 'l2beat_id', 'website', 'colors', 'symbol', 'similar_chains', 'links', 'logo', 'active_tabs', 'evm_chain_id', 'rhino_listed', 'tab_status', 'description', 'bucket', 'chain_name', 'stack', 'raas', 'name', 'block_explorer', 'l2beat_stage', 'origin_key', 'enable_contracts', 'maturity', 'da_layer', 'l2beat_l

2025-11-03 16:54:13,445 INFO root Attempt 1: Using service account credentials
2025-11-03 16:54:13,446 WARNING root Service account init failed on attempt 1: [Errno 2] No such file or directory: '/Users/michaelsilberling/Documents/bq/oplabs-tools-data-0842f34ceb69.json'
2025-11-03 16:54:13,447 INFO root Attempt 1: Using ADC/OIDC login


DataFrame columns: {'technology', 'excluded_metrics', 'gas_token', 'twitter', 'rhino_naming', 'supported_metrics', 'l2beat_id', 'website', 'colors', 'symbol', 'similar_chains', 'links', 'logo', 'evm_chain_id', 'rhino_listed', 'tab_status', 'description', 'bucket', 'chain_name', 'stack', 'raas', 'name', 'block_explorer', 'l2beat_stage', 'origin_key', 'enable_contracts', 'maturity', 'da_layer', 'l2beat_link', 'block_explorers', 'url_key', 'deployment', 'chain_type', 'ecosystem', 'caip2', 'company', 'launch_date', 'purpose'}
Schema columns: {'technology', 'twitter', 'excluded_metrics', 'gas_token', 'rhino_naming', 'supported_metrics', 'l2beat_id', 'website', 'colors', 'symbol', 'similar_chains', 'links', 'logo', 'active_tabs', 'evm_chain_id', 'rhino_listed', 'tab_status', 'description', 'bucket', 'chain_name', 'stack', 'raas', 'name', 'block_explorer', 'l2beat_stage', 'origin_key', 'enable_contracts', 'maturity', 'da_layer', 'l2beat_link', 'block_explorers', 'url_key', 'deployment', 'chai

2025-11-03 16:54:16,388 INFO root Attempt 1: Using service account credentials
2025-11-03 16:54:16,389 WARNING root Service account init failed on attempt 1: [Errno 2] No such file or directory: '/Users/michaelsilberling/Documents/bq/oplabs-tools-data-0842f34ceb69.json'
2025-11-03 16:54:16,389 INFO root Attempt 1: Using ADC/OIDC login


DataFrame columns: {'technology', 'excluded_metrics', 'gas_token', 'twitter', 'rhino_naming', 'supported_metrics', 'l2beat_id', 'website', 'colors', 'symbol', 'similar_chains', 'links', 'logo', 'evm_chain_id', 'rhino_listed', 'tab_status', 'description', 'bucket', 'chain_name', 'stack', 'raas', 'name', 'block_explorer', 'l2beat_stage', 'origin_key', 'enable_contracts', 'maturity', 'da_layer', 'l2beat_link', 'block_explorers', 'url_key', 'deployment', 'chain_type', 'ecosystem', 'caip2', 'company', 'launch_date', 'purpose'}
Schema columns: {'technology', 'twitter', 'excluded_metrics', 'gas_token', 'rhino_naming', 'supported_metrics', 'l2beat_id', 'website', 'colors', 'symbol', 'similar_chains', 'links', 'logo', 'active_tabs', 'evm_chain_id', 'rhino_listed', 'tab_status', 'description', 'bucket', 'chain_name', 'stack', 'raas', 'name', 'block_explorer', 'l2beat_stage', 'origin_key', 'enable_contracts', 'maturity', 'da_layer', 'l2beat_link', 'block_explorers', 'url_key', 'deployment', 'chai

In [ ]:
# Post to Dune API
try:
    if not combined_gtp_df.empty:
        d.write_dune_api_from_pandas(
            combined_gtp_df, "growthepie_l2_activity", "L2 Usage Activity from GrowThePie"
        )
    if not gtp_meta_api.empty:
        d.write_dune_api_from_pandas(
            gtp_meta_api, "growthepie_l2_metadata", "L2 Metadata from GrowThePie"
        )
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Exception type: {type(e).__name__}")
    import traceback
    print("Traceback:")
    print(traceback.format_exc())
# d.write_dune_api_from_pandas(
#     l2b_enriched_df, "l2beat_l2_activity", "L2 Usage Activity from L2Beat"
# )
# d.write_dune_api_from_pandas(
#     l2b_monthly_df,
#     "l2beat_l2_activity_monthly",
#     "Monthly L2 Usage Activity from L2Beat",
# )
# d.write_dune_api_from_pandas(
#     l2b_weekly_df, "l2beat_l2_activity_weekly", "Weekly L2 Usage Activity from L2Beat"
# )
# d.write_dune_api_from_pandas(
#     l2beat_meta, "l2beat_l2_metadata", "L2 Metadata from L2Beat"
# )

In [ ]:
# Copy to Dune
print('upload bq to dune')
sql = '''
SELECT *
FROM `views.daily_l2beat_breakdown`
WHERE onchain_value_usd > 0
AND dt_day >= DATE_SUB(CURRENT_DATE(), INTERVAL 365 DAY)
'''
bq_df = bqu.run_query_to_df(sql)

dune_table_name = 'daily_l2beat_breakdown'
d.write_dune_api_from_pandas(bq_df, dune_table_name,table_description = dune_table_name)